In [18]:
import os,datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

time_str = (datetime.datetime.now()).strftime("%Y%m%d")
pre_fix = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_Rider/VOC2007/'
xml_dir = pre_fix + '/Annotations/'
image_dir = pre_fix + '/JPEGImages/'

generated_image_dir = pre_fix + '/images/'
generated_xml_dir = pre_fix + '/xmls/'

#os.system('mkdir -p ' + xml_dir)
#os.system('mkdir -p ' + image_dir)
os.system('mkdir -p ' + generated_image_dir)
os.system('mkdir -p ' + generated_xml_dir)

SEED_XML_DIR = xml_dir
SEED_IMG_DIR = image_dir
GENE_IMG_DIR = generated_image_dir
GENE_XML_DIR = generated_xml_dir


# RT:RightTop 
# LB:LeftBottom 
# bbox: [xmin, xax, ymin, ymax]
def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]

def get_obj_from_xml(xml_name):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]

def get_obj_from_image_file(file, bbox):
    img = cv2.imread(file)
    img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
   #print(img_obj.shape)
    return img_obj

def get_bboxes_from_etree(etree):
    root = etree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def past_to_background_from_image_file(file, bboxes, background_img_array, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    if(img.shape != background_img_array.shape):
       #print('shape not match')
        return
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    cv2.imwrite(GENE_IMG_DIR+file.split('/')[-1], background_img_array)
    return 
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object

def generate_new_xmlobj_simple(xmlobj_old, new_bbox):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text
    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text
    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(int(new_bbox[0]))
    tag_ymin.text = str(int(new_bbox[2]))
    tag_xmax.text = str(int(new_bbox[1]))
    tag_ymax.text = str(int(new_bbox[3]))
    return element_object
    
def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree
    

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img
    
def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def generate_new_position(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    #print(new_position)
    return new_position

def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
def generate_union_bbox(matrix1, matrix2): #xin,xmax,ymin,ymax
    img_obj_shape = (int(max(matrix1[3], matrix2[3])-min(matrix1[2], matrix2[2])), 
                int(max(matrix1[1], matrix2[1])-min(matrix1[0], matrix2[0])))
    return img_obj_shape

def generate_union_bbox_shape(matrix_list): #xin,xmax,ymin,ymax
    matrix_array = np.array(matrix_list)
    img_obj_shape = (int(max(matrix_array[:,3]) - min(matrix_array[:,2])),
                     int(max(matrix_array[:,1]) - min(matrix_array[:,0])))
    return img_obj_shape

def get_union_bbox_left_top(matrix_list): #xin,xmax,ymin,ymax
    matrix_array = np.array(matrix_list)
    left_top = (int(min(matrix_array[:,0])),
                     int(min(matrix_array[:,2])))
    return left_top
def generate_union_bbox_(matrix_list):
    img_obj_shape = generate_union_bbox_shape(matrix_list)
    left_top = get_union_bbox_left_top(matrix_list)
    union_bbox = [int(left_top[0]), 
                  int(left_top[0]+int(img_obj_shape[1])),
                  int(left_top[1]),
                  int(left_top[1]+int(img_obj_shape[0]))]
    return union_bbox

def generate_new_bbox_within_distance(img_size, img_obj_size, old_bbox, scale=2.5):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    #array_y = np.arange(int(img_size[0] - img_obj_size[0])/3, int(img_size[0] - img_obj_size[0])/3*2)
    array_y = np.arange(max(0, int((old_bbox[3]+old_bbox[2])/2 - scale*img_obj_size[0])),
                        min(int((old_bbox[3]+old_bbox[2])/2 + scale*img_obj_size[0]), int(img_size[0] - img_obj_size[0])))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox

def check_before_insert(img_size, img_obj_size, etree, union_bbox):
    new_bbox = generate_new_bbox_within_distance(img_size, img_obj_size, union_bbox)
    bboxes = get_bboxes_from_etree(etree)
    retry_times = 0
    while(not check_bbox(new_bbox, bboxes)):
       #print('new_bbox not suitable, retry...')
        retry_times = retry_times + 1
        if(retry_times > 100):
            return (False, False)
        new_bbox = generate_new_bbox_within_distance(img_size, img_obj_size, union_bbox)
    return (new_bbox[0], new_bbox[2])
    
def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    retry_times = 0
    while(not check_bbox(new_bbox, bboxes)):
       #print('new_bbox not suitable, retry...')
        retry_times = retry_times + 1
        if(retry_times > 50):
            return False, False
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    #print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, etree)
    
    return new_pil_img, new_xml_etree

def do_inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree, insert_point):
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, insert_point, obj_element, etree)    
    return new_pil_img, new_xml_etree

def inset_dobule_obj_to_an_image_and_xml(img_obj1, img_obj2, img, obj_element1, obj_element2, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    retry_times = 0
    while(not check_bbox(new_bbox, bboxes)):
       #print('new_bbox not suitable, retry...')
        retry_times = retry_times + 1
        if(retry_times > 50):
            return False, False
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    #print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, etree)
    
    return new_pil_img, new_xml_etree

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0):
            return False
    return True
def caculate_move_vector(bbox, new_point):
    original_xmin, original_ymin = bbox[0], bbox[2]
    diffx, diffy = new_point[0] - original_xmin, new_point[1] - original_ymin
    return (diffx, diffy)

def rotate_img(img, thealta):
    (h_, w_) = img.shape[:2]
    point_list = [(0, 0), (0, h_), (w_, h_), (w_, 0)]
    center = (w_ // 2, h_ // 2)
    roted_point_list = [((point[0]-center[0])*np.cos(np.pi*thealta/180) 
                         - (point[1]-center[1])*np.sin(np.pi*thealta/180) 
                         + center[0], (point[0]-center[0])*np.sin(np.pi*thealta/180) 
                         + (point[1]-center[1])*np.cos(np.pi*thealta/180) + center[1]) for point in point_list]
    #print(roted_point_list)

    temp = np.zeros((2, 4))
    temp[0] = [roted_point[0] for roted_point in roted_point_list]
    temp[1] = [roted_point[1] for roted_point in roted_point_list]
    (xmin, xmax, ymin, ymax) = (np.min(temp[0]), np.max(temp[0]), np.min(temp[1]), np.max(temp[1]))
   #print('xmin: {0}, xmax: {1}, ymin: {2}, ymax: {3}'.format(xmin, xmax, ymin, ymax))

    roted_h, roted_w = ymax - ymin, xmax - xmin
   #print('roted_h: {0}, roted_w: {1}'.format(roted_w, roted_h))
    
    top_bottom, left_right = int((roted_h - h_)/2), int((roted_w - w_)/2)
   #print('top_bottom:{0}, left_right_:{1}'.format(top_bottom, left_right))
    padding = lambda arg : max(arg, 0)
    #dst = cv2.copyMakeBorder(img, padding(top_bottom), padding(top_bottom), padding(left_right), 
    #padding(left_right), cv2.BORDER_CONSTANT)
    dst = cv2.copyMakeBorder(img, padding(top_bottom), padding(top_bottom), padding(left_right), 
                             padding(left_right), cv2.BORDER_CONSTANT)
    
    (h, w) = dst.shape[:2]   
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, thealta, 1.0)
    rotated_img = cv2.warpAffine(dst, M, (w, h))
    if(top_bottom < 0):
        top_bottom = abs(top_bottom)
        rotated_img = rotated_img[top_bottom:-top_bottom, :, :]
    if(left_right < 0):
        left_right = abs(left_right)
        rotated_img = rotated_img[:, left_right:-left_right, :]
    return rotated_img

def get_fliped_bbox(bbox, img_w):
    return (img_w-bbox[0], img_w-bbox[1], bbox[2], bbox[3])

def generate_etree(etree_old):
    root_old = etree_old.getroot()    
    root = ET.Element('annotation')
    root.append(root_old.find('folder'))
    root.append(root_old.find('filename'))
    root.append(root_old.find('source'))
    root.append(root_old.find('size'))
    root.append(root_old.find('segmented'))    
    tree = ET.ElementTree(root)    
    return tree
def get_intercouse_box(bbox_a, bbox_b):
    xmin_ = max(bbox_a[0], bbox_b[0])
    xmax_ = min(bbox_a[1], bbox_b[1])
    ymin_ = max(bbox_a[2], bbox_b[2])
    ymax_ = min(bbox_a[3], bbox_b[3])
    if((xmax_ - xmin_ < 20) or (ymax_ - ymin_ < 20)):
        return False
    return [xmin_, xmax_, ymin_, ymax_]

def generate_new_bbox_within_distance(img_size, img_obj_size, old_bbox, scale=2.5):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    #array_y = np.arange(int(img_size[0] - img_obj_size[0])/3, int(img_size[0] - img_obj_size[0])/3*2)
    array_y = np.arange(max(0, int((old_bbox[3]+old_bbox[2])/2 - scale*img_obj_size[0])),
                        min(int((old_bbox[3]+old_bbox[2])/2 + scale*img_obj_size[0]), int(img_size[0] - img_obj_size[0])))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox

############################# for expand obj target-intercouses (overlaps) ##################################
LOOP_TIMES = 3
REPEAT_TIMES = 2
seed_xml_names = os.listdir(SEED_XML_DIR)
seed_xml_names.sort()
cls_to_rotate = ['rider-mo', 'rider-bi']

for i in range (LOOP_TIMES):
    for xml_name in seed_xml_names:
        img_name = SEED_IMG_DIR + xml_name[:-3] + 'jpg'
        xml_name = SEED_XML_DIR + xml_name
       #print(img_name)
       #print(xml_name)
        
        tree=ET.parse(xml_name)
        img = cv2.imread(img_name)
        objs = get_obj_from_xml(xml_name)
        bboxes = get_bboxes_from_etree(tree)
        flag_found = False
        i = 0
        #------------------------------------------------------------
        bboxes_target = {}
        for obj_element in objs:
            bbox = get_bbox_from_xmlobj(obj_element)           
            img_obj = get_obj_from_image_file(img_name, bbox)

            cls = get_cls_from_xmlobj(obj_element)
            if (cls in cls_to_rotate):
                bboxes_target[(i)] = bbox
            i = i + 1
        id_dic = {}
        ##------------------------------------------------##
        target_keys = bboxes_target.keys()
        target_bboxes = bboxes_target.values()
        target_keys_list = list(target_keys)
        target_bboxes_list = list(target_bboxes)
        #print(target_keys_list)
        #print(target_bboxes_list)
        _i = 0
        for _bbx in target_bboxes_list:
            _i = _i + 1
            id_list = []
            #for __bbx in target_bboxes_list[_i:]: ##forward
            for __bbx in target_bboxes_list:
                if(_bbx == __bbx): ##the same id
                    continue
                if(IOU(_bbx, __bbx) > 0.1):
                    id_list.append(target_keys_list[target_bboxes_list.index(__bbx)])
            id_dic[(target_keys_list[_i-1])] = id_list
        #print('id_dic:{0}'.format(id_dic))
        for k__, v__ in id_dic.items():
            for v_ in v__:
                if(len(id_dic[v_]) == 0):
                    continue
                id_dic[k__] = id_dic[k__] + id_dic[v_]
                id_dic[k__] = list(set(id_dic[k__]))
                id_dic[k__].sort()
        #print('id_dic:{0}'.format(id_dic))

        '''
        for __k in list(id_dic.keys()):
            if(len(id_dic[__k]) == 0):
                id_dic.pop(__k)       
        #print('id_dic:{0}'.format(id_dic))
        '''
        
        v_list = []
        id_dic_refine = {}
        for __k, __v in id_dic.items():
            if(__k not in v_list):
                v_list = v_list + __v
                #print('k:{0};v_list:{1}'.format(__k, v_list))
                id_dic_refine[(__k)] = __v
                
        id_dic = id_dic_refine
       #print('id_dic:{0}'.format(id_dic))

        #continue;
        
        FLAG = False
        for i in range(REPEAT_TIMES):
            for kk, vv in id_dic.items():
                if(len(vv) == 0):
                    templist = [bboxes[kk]]
                    ##templist.append(bboxes[kk])
                    img_obj_shape = generate_union_bbox_shape(templist)
                    union_bbox = generate_union_bbox_(templist)
                    (xmin, ymin) = check_before_insert(img.shape[:2], img_obj_shape, tree, union_bbox)
                    if(xmin == False) or (ymin == False):
                       #print("give up, try next bbox")
                        continue
                    (diffx, diffy) = caculate_move_vector(union_bbox, (xmin, ymin))
                   #print(diffx)
                    new_left_top = (int(bboxes[kk][0] + diffx), int(bboxes[kk][2] + diffy))
                    img_obj = get_obj_from_image_file(img_name, bboxes[kk])
                    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, new_left_top, objs[kk], tree)
                    if((new_pil_img == False) or (new_xml_etree == False)):
                        continue
                    tree = new_xml_etree
                    img = np.array(new_pil_img)
                    FLAG = True
                elif(len(vv) >= 1):
                    templist = [bboxes[v] for v in vv]
                    ##templist.append(bboxes[kk])
                    img_obj_shape = generate_union_bbox_shape(templist)
                    union_bbox = generate_union_bbox_(templist)
                    (xmin, ymin) = check_before_insert(img.shape[:2], img_obj_shape, tree, union_bbox)
                    if(xmin == False) or (ymin == False):
                       #print("give up, try next bbox")
                        continue
                    (diffx, diffy) = caculate_move_vector(union_bbox, (xmin, ymin))
                    for v in vv:
                        new_left_top = (int(bboxes[v][0] + diffx), int(bboxes[v][2] + diffy))
                        img_obj = get_obj_from_image_file(img_name, bboxes[v])
                        new_pil_img, new_xml_etree = past_and_insert(img_obj, img, new_left_top, objs[v], tree)
                        if((new_pil_img == False) or (new_xml_etree == False)):
                            continue
                        tree = new_xml_etree
                        img = np.array(new_pil_img)
                        FLAG = True
        ## save xml and img
        if(FLAG != True):
            continue
        time_mark = datetime.datetime.now()
        time_str = time_mark.strftime("%Y%m%d%H%M%S%f_")
        new_pil_img = Image.fromarray(img[:,:,(2,1,0)])
        new_pil_img.save(GENE_IMG_DIR + time_str + img_name.split('/')[-1])
        tree.write(GENE_XML_DIR + time_str + xml_name.split('/')[-1])
        
        
############################# for flip ##################################
SEED_XML_DIR = generated_xml_dir
SEED_IMG_DIR = generated_image_dir
GENE_IMG_DIR = generated_image_dir
GENE_XML_DIR = generated_xml_dir

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_xml_names.sort()

print(len(seed_xml_names))
for xml_name in seed_xml_names:
        img_name = SEED_IMG_DIR + xml_name[:-3] + 'jpg'
        xml_name = SEED_XML_DIR + xml_name
       #print(img_name)
       #print(xml_name)
        
        tree=ET.parse(xml_name)
        new_tree = generate_etree(tree)
        img = cv2.imread(img_name)
       #print(img.shape)
        
        objs = get_obj_from_xml(xml_name)
        for obj_element in objs:
            bbox = get_bbox_from_xmlobj(obj_element)
            new_bbox = get_fliped_bbox(bbox, img.shape[1])
            #print(bbox)
            #print(new_bbox)
            new_obj_element = generate_new_xmlobj_simple(obj_element, new_bbox)
            new_tree = insert_to_xml(new_obj_element, new_tree)
        time_mark = datetime.datetime.now()
        time_str = time_mark.strftime("%Y%m%d%H%M%S%f_")
        #new_tree.write(GENE_XML_DIR + 'fliped_' + time_str + xml_name.split('/')[-1])
        new_tree.write(GENE_XML_DIR + 'fliped_'  + xml_name.split('/')[-1])
        h_flip = cv2.flip(img, 1)
        #cv2.imwrite(GENE_IMG_DIR + 'fliped_' + time_str + img_name.split('/')[-1], h_flip)
        cv2.imwrite(GENE_IMG_DIR + 'fliped_'  + img_name.split('/')[-1], h_flip)
        
        
        
############################# for crop flip ##################################        
SEED_XML_DIR = generated_xml_dir
SEED_IMG_DIR = generated_image_dir
GENE_IMG_DIR = generated_image_dir
GENE_XML_DIR = generated_xml_dir

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_xml_names.sort()

croped_shape = [480, 640]
LOOPTIMES = 1
for L in range(LOOPTIMES):
    for xml_name in seed_xml_names:
        img_name = SEED_IMG_DIR + xml_name[:-3] + 'jpg'
        xml_name = SEED_XML_DIR + xml_name
        #print(img_name)
        #print(xml_name)

        etree=ET.parse(xml_name)
        img = cv2.imread(img_name)
        croped_shape = [int(img.shape[0]/1.5), int(img.shape[1]/2)]
        objs = get_obj_from_xml(xml_name)
        bboxes = get_bboxes_from_etree(tree)

        ## 0.generate new empty element tree
        new_etree = generate_etree(etree)
        root = new_etree.getroot()
        xml_obj_size = root.find('size')
        xml_obj_size.find('width').text = str(croped_shape[1])
        xml_obj_size.find('height').text = str(croped_shape[0])
        #new_etree.write('/data/test.xml')

        ## 1.generate crop box
        new_bbox = generate_new_bbox(img.shape[:2], croped_shape)
        img_obj = get_obj_from_image_file(img_name, new_bbox)
        (diffx, diffy) = caculate_move_vector(new_bbox, (0, 0))

        ## 2.find the obj that in crop box
        new_objs = []
        for obj_element in objs:
            bbox = get_bbox_from_xmlobj(obj_element)
            inter_bbox = get_intercouse_box(bbox, new_bbox)
            if(inter_bbox == False):
                continue            
            inter_bbox_trans = [int(inter_bbox[0] + diffx), int(inter_bbox[1] + diffx), 
                                int(inter_bbox[2] + diffy), int(inter_bbox[3] + diffy)]
            cls = get_cls_from_xmlobj(obj_element)
            new_xmlobj = generate_new_xmlobj(obj_element, (inter_bbox_trans[0], inter_bbox_trans[2]), 
                                (int(inter_bbox[3] - inter_bbox[2]), int(inter_bbox[1] - inter_bbox[0])))
            new_objs.append(new_xmlobj)
        if(len(new_objs) > 0):
            for i in new_objs:
                root.append(i)
        else:
            continue   

        ## save xml and img
        time_mark = datetime.datetime.now()
        time_str = time_mark.strftime("%Y%m%d%H%M%S%f_")
        cv2.imwrite(GENE_IMG_DIR + 'croped_' + img_name.split('/')[-1], img_obj)    
        new_etree.write(GENE_XML_DIR + 'croped_' + xml_name.split('/')[-1])

############################# for rescale ##################################              
SEED_XML_DIR = generated_xml_dir
SEED_IMG_DIR = generated_image_dir
GENE_IMG_DIR = generated_image_dir
GENE_XML_DIR = generated_xml_dir

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_xml_names.sort()

for i in [0.3]:
    x_shrink_scale = i
    y_shrink_scale = 0
    padding = 127.5
    for xml in seed_xml_names:

        # generate new jpg image file
        img_file = xml[:-4] + '.jpg'
        img_path = SEED_IMG_DIR + img_file
       #print(img_path)
        img = cv2.imread(img_path)
        array = np.zeros_like(img)
        array.fill(padding)   
        new_w = int(img.shape[1]*(1-x_shrink_scale))
        new_h = int(img.shape[0]*(1-y_shrink_scale))
        shrink_width = img.shape[1] - new_w
        shrink_heigt = img.shape[0] - new_h
        #img = cv2.resize(img, (new_w, img.shape[0]))
        #array[:, int(shrink_width/2):int(shrink_width/2) + new_w] = img
        img = cv2.resize(img, (new_w, new_h))
        array[int(shrink_heigt/2):int(shrink_heigt/2) + new_h, int(shrink_width/2):int(shrink_width/2) + new_w] = img
        cv2.imwrite(GENE_IMG_DIR + "ge_shrink_scale_" + str(x_shrink_scale) + "_" + str(y_shrink_scale) + "_" + str(padding) + "_"+ img_file, array)

        # generate new anotation xml file    
        xml_path = SEED_XML_DIR + xml
        in_file = open(xml_path)
        tree=ET.parse(in_file)
        root = tree.getroot()

        for obj in root.iter('object'):
            '''
            difficult = obj.find('difficult').text
            cls_ = obj.find('name').text
            if cls_ not in classes_ or int(difficult)==1:
                continue
            '''
            xmlbox = obj.find('bndbox')
            xmin, xmax = int(xmlbox.find('xmin').text), int(xmlbox.find('xmax').text)
            ymin, ymax = int(xmlbox.find('ymin').text), int(xmlbox.find('ymax').text)
            xmlbox.find('xmin').text = str(int(xmin*(1-x_shrink_scale) + shrink_width/2))
            xmlbox.find('xmax').text = str(int(xmax*(1-x_shrink_scale) + shrink_width/2))
            xmlbox.find('ymin').text = str(int(ymin*(1-y_shrink_scale) + shrink_heigt/2))
            xmlbox.find('ymax').text = str(int(ymax*(1-y_shrink_scale) + shrink_heigt/2))
        tree.write(GENE_XML_DIR + "ge_shrink_scale_" + str(x_shrink_scale) + "_" + str(y_shrink_scale) +  "_" + str(padding) +"_" + xml)
        
for i in [0.3]:
    x_shrink_scale = 0
    y_shrink_scale = i
    padding = 127.5
    for xml in seed_xml_names:

        # generate new jpg image file
        img_file = xml[:-4] + '.jpg'
        img_path = SEED_IMG_DIR + img_file
        img = cv2.imread(img_path)
        array = np.zeros_like(img)
        array.fill(padding)   
        new_w = int(img.shape[1]*(1-x_shrink_scale))
        new_h = int(img.shape[0]*(1-y_shrink_scale))
        shrink_width = img.shape[1] - new_w
        shrink_heigt = img.shape[0] - new_h
        #img = cv2.resize(img, (new_w, img.shape[0]))
        #array[:, int(shrink_width/2):int(shrink_width/2) + new_w] = img
        img = cv2.resize(img, (new_w, new_h))
        array[int(shrink_heigt/2):int(shrink_heigt/2) + new_h, int(shrink_width/2):int(shrink_width/2) + new_w] = img
        cv2.imwrite(GENE_IMG_DIR + "ge_shrink_scale_" + str(x_shrink_scale) + "_" + str(y_shrink_scale) + "_" + str(padding) + "_"+ img_file, array)

        # generate new anotation xml file    
        xml_path = SEED_XML_DIR + xml
        in_file = open(xml_path)
        tree=ET.parse(in_file)
        root = tree.getroot()

        for obj in root.iter('object'):
            '''
            difficult = obj.find('difficult').text
            cls_ = obj.find('name').text
            if cls_ not in classes_ or int(difficult)==1:
                continue
            '''
            xmlbox = obj.find('bndbox')
            xmin, xmax = int(xmlbox.find('xmin').text), int(xmlbox.find('xmax').text)
            ymin, ymax = int(xmlbox.find('ymin').text), int(xmlbox.find('ymax').text)
            xmlbox.find('xmin').text = str(int(xmin*(1-x_shrink_scale) + shrink_width/2))
            xmlbox.find('xmax').text = str(int(xmax*(1-x_shrink_scale) + shrink_width/2))
            xmlbox.find('ymin').text = str(int(ymin*(1-y_shrink_scale) + shrink_heigt/2))
            xmlbox.find('ymax').text = str(int(ymax*(1-y_shrink_scale) + shrink_heigt/2))
        tree.write(GENE_XML_DIR + "ge_shrink_scale_" + str(x_shrink_scale) + "_" + str(y_shrink_scale) +  "_" + str(padding) +"_" + xml)
        
        
os.system('python3 /datasets/labelImg/labelImg.py &')

861


0